In [2]:
import os
import sys
import glob
import json
import requests
import pandas as pd
import pandas_gbq as gbq
from pathlib import Path
from IPython.core.debugger import set_trace
from tqdm.auto import tqdm
tqdm.pandas()

# Bigquery config

In [3]:
proj_id = 'nlp-project-253707'
table_id = 'news_dataset.news'

# Tagged news from Bigquery

In [181]:
sql = '''
SELECT url
FROM `nlp-project-253707.news_dataset.news`
'''

tagged_news = gbq.read_gbq(sql, project_id=proj_id)
tagged_news_url = list(tagged_news.url)

# check uniqueness of urls
assert len(tagged_news_url) == len(set(tagged_news_url)), 'url duplicated'

In [182]:
len(tagged_news)

18000

# Listing texts & urls to tag

In [168]:
def extract_to_tag(urls_asis, n_tag=500, where='newsdata/downloaded/*.json'):
    '''
    n_tag=500: uclassify가 허용하는 최대 text수가 500개임
    '''
    
    urls = []
    texts = []
    
    n = 0
    for fname in glob.glob(where):
        js = json.loads(Path(fname).read_text())

        if js['url'] not in urls_asis:        
            texts.append(js['text'])
            urls.append(js['url'])
            n += 1
            print('\r{}'.format(n), end='')
            if n == n_tag: break
                
    return texts, urls

In [169]:
texts, urls = extract_to_tag(tagged_news_url)

500

# Classification using uClassify
https://www.uclassify.com

In [170]:
apikeys = {
            'gem763': 'YbEhDr9hAYmd', 
    'projectester01': 'PUcCZkyO5wPs', # google 로그인해야함
    'projectester02': 'f2ISRvxphKf2',
    'projectester03': 'ebAUBLTIcdna',
    'projectester04': 'UTuQWyexBdOP',
    'projectester05': 'nfsPpO8ivVaR',
    'projectester06': '4659gHVFX8CQ',
    'projectester07': 'UljdQT1tzeVs',
    'projectester08': 'RWh9JGLYRho5',
    'projectester09': 'EHjhm6nONJv4',
    'projectester10': 'QZlAZF62SaEc',
    'projectester11': 'zxJQoBwzi1Bw',
    'projectester12': 'uZRjYK6Vx5aC',
    'projectester13': 'Vjk2T02S0Mvz',
    'projectester14': 'Z46sXa4fYKyO',
    'projectester15': 'EwLxqluy7WcY',
    'projectester16': 'NCVDU2DMxEXB',
    'projectester17': 'xUMOMxOIoWUY',
    'projectester18': 'B93XpEXzZsfa',
    'projectester19': 'vLuLVnQ4sCmM',
    'projectester20': 'ACIHqZxBXEBO',
}

In [171]:
def uclassify(_id, texts):
    headers = {'Authorization':'Token {}'.format(apikeys[_id])}
    endpoint = 'https://api.uclassify.com/v1/uclassify/IAB Taxonomy v2/classify'
    data = json.dumps({'texts': texts})

    return requests.post(endpoint, headers=headers, data=data)


def uclassify_response_to_dataframe(u_resp, urls, texts):
    records = []
    updated_at = str(pd.Timestamp.utcnow())

    for i, resp in enumerate(tqdm(u_resp.json())):
        cl = max(resp['classification'], key=lambda cand:cand['p'])    
        records.append({
            'url': urls[i], 
            'text': texts[i], 
            'uclass': cl['className'].split('_')[0], 
            'prob': cl['p'], 
            'updated_at': updated_at, 
        })

    return pd.DataFrame(records)

In [172]:
response = uclassify('projectester18', texts)

In [173]:
df = uclassify_response_to_dataframe(response, urls, texts); df

,prob,text,uclass,updated_at,url
0,0.999896,"When it comes to 5G stocks, I prefer a simple,...",automotive,2019-09-29 08:33:36.074966+00:00,https://finance.yahoo.com/news/5g-stocks-bigge...
1,0.619169,"Sheldon Whitehouse, a Democrat, represents Rho...",news and politics,2019-09-29 08:33:36.074966+00:00,https://washingtonpost.com/opinions/the-suprem...
2,0.576536,Despite beating expectations in its first quar...,personal finance,2019-09-29 08:33:36.074966+00:00,https://news.yahoo.com/why-slacks-become-a-vic...
3,0.987763,Image copyright EPA Image caption Germany's Ur...,news and politics,2019-09-29 08:33:36.074966+00:00,https://bbc.com/news/world-europe-49646809
4,0.721777,UK car production declined for the 14th consec...,news and politics,2019-09-29 08:33:36.074966+00:00,https://independent.co.uk/news/business/news/u...
5,0.768826,"After backing Bernie Sanders in 2016, the Work...",news and politics,2019-09-29 08:33:36.074966+00:00,https://thedailybeast.com/working-families-par...
6,0.708656,Charli XCX has opened up about the stigma surr...,pop culture,2019-09-29 08:33:36.074966+00:00,https://independent.co.uk/life-style/health-an...
7,0.985412,New wave singer behind hits including Good Tim...,music and audio,2019-09-29 08:33:36.074966+00:00,https://theguardian.com/music/2019/sep/16/ric-...
8,0.980354,"On Wednesday, the U.S. Federal Reserve conclud...",personal finance,2019-09-29 08:33:36.074966+00:00,https://realmoney.thestreet.com/investing/fixe...
9,0.248107,“We don’t usually look to Washington to solve ...,business and finance,2019-09-29 08:33:36.074966+00:00,https://politico.com/news/2019/09/28/ransomwar...


# Bigquery update

In [174]:
gbq.to_gbq(df, table_id, project_id=proj_id, if_exists='append')

1it [00:00,  1.78it/s]


In [180]:
df.iloc[4].text

'UK car production declined for the 14th consecutive month in July, slumping by 10.6 per cent from a year ago, as key markets in the EU and Asia remained weak, according to the Society of Motor Manufacturers and Traders (SMMT).\n\nThe last time the sector suffered such a prolonged downturn predated even the last recession, with production sliding for 14 months in a row between 2000 and 2001, the SMMT said.\n\nIts figures released on Thursday also showed that exports fell 14.6 per cent in July but still eight in every 10 cars were shipped overseas, underscoring the importance of foreign appetite for British-made cars.\n\nWe’ll tell you what’s true. You can form your own view. From 15p €0.18 $0.18 USD 0.27 a day, more exclusives, analysis and extras.\n\nMike Hawes, SMMT chief executive, called another month of decline in car manufacturing “a serious concern”.\n\n“The sector is overwhelmingly reliant on exports, and the global headwinds are strong, with escalating trade tensions, softenin

In [137]:
sql = '''
SELECT *
FROM `nlp-project-253707.news_dataset.news`
'''

test = gbq.read_gbq(sql, project_id=proj_id)

In [218]:
test

,prob,text,uclass,updated_at,url
0,0.292673,\n\nChris Sharp of the Fairfax County Police D...,pets,2019-09-26,https://washingtonpost.com/local/public-safety...
1,0.472381,Dir: Roger Hinze and Michael William Miles. Fe...,movies,2019-09-26,https://independent.co.uk/arts-entertainment/f...
2,0.621441,"Horror film ""IT Chapter Two"" stayed atop the N...",movies,2019-09-26,https://businesstimes.com.sg/life-culture/it-s...
3,0.993574,The 2019 Emmys are officially here.\n\nThe 71s...,movies,2019-09-26,https://yahoo.com/entertainment/emmys-2019-win...
4,0.293006,Eddie Murphy in Netflix’s Dolemite Is My Name ...,movies,2019-09-26,https://vulture.com/2019/09/tiff-screening-ven...
5,0.961838,This article originally appeared on VICE US.\n...,movies,2019-09-26,https://vice.com/en_asia/article/kz4bvw/mike-f...
6,0.434689,Image copyright Alamy Image caption Movie rema...,movies,2019-09-26,https://bbc.co.uk/news/newsbeat-49755700
7,0.670388,Angelina Jolie is a busy Hollywood actress and...,movies,2019-09-26,https://yahoo.com/entertainment/angelina-jolie...
8,0.511823,MEXICO CITY (Reuters) - Mexico's president on ...,sports,2019-09-26,https://businessinsider.com/mexico-president-s...
9,0.966641,MEXICO CITY (Reuters) - Mexican President Andr...,sports,2019-09-26,https://reuters.com/article/us-mexico-constell...


In [220]:
test[test.duplicated('url')]

,prob,text,uclass,updated_at,url
807,0.941261,But what if the way we think about paying our ...,business and finance,2019-09-26,https://politico.com/interactives/2019/how-to-...
